In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#X_train = X_train - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]
#X_test = X_test - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]

X_train_full, y_train_full = X_train, y_train

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ferbi\appd

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_inputs = X_train.shape[1]
n_hidden1 = 1024
n_hidden2 = 1024
n_hidden3 = 1024
n_hidden4 = 1024
n_hidden5 = 1024
n_outputs = 10

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4", activation=tf.nn.relu)
    hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5", activation=tf.nn.relu)
    logits = neuron_layer(hidden5, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
hidden3_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/kernel:0")
hidden4_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/kernel:0")
hidden5_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")
    
def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 100000*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 8
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden3_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden4_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden5_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(hidden3_W), \
                                  tf.count_nonzero(hidden4_W), \
                                  tf.count_nonzero(hidden5_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden3_W) < epsVal, hidden3_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden4_W) < epsVal, hidden4_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden5_W) < epsVal, hidden5_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

reg_summary = tf.summary.scalar('Count_Reg', total_regvalue)
loss_summary = tf.summary.scalar('Loss_Total', loss_total)
err_summary = tf.summary.scalar('Class_Err_Perc', (1-accuracy)*100)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, reg_loss_val, lambda1, acc_val = sess.run([loss_total, total_regvalue, lambdaParam, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        base_loss_val = loss_val - lambda1*reg_loss_val
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation losses: {:.4f}, {:.4f}, {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, base_loss_val, lambda1*reg_loss_val, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20190129210937.ckpt") # or better, use save_path
    n_batches = len(X_test) // 2000
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [ ]:
for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
    for i in [1, 2, 3]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -4.0
Now: 20240323185446
0	Validation losses: 0.4093, 415.2361, 415.6453	Best Loss: 415.6453 (0)	Accuracy: 84.48%
1	Validation losses: 0.3415, 407.4875, 407.8290	Best Loss: 407.8290 (0)	Accuracy: 87.86%
2	Validation losses: 0.3630, 401.5397, 401.9027	Best Loss: 401.9027 (0)	Accuracy: 86.78%
3	Validation losses: 0.3404, 396.1848, 396.5252	Best Loss: 396.5252 (0)	Accuracy: 87.48%
4	Validation losses: 0.3206, 390.8895, 391.2101	Best Loss: 391.2101 (0)	Accuracy: 88.02%
5	Validation losses: 0.3981, 385.8954, 386.2935	Best Loss: 386.2935 (0)	Accuracy: 85.94%
6	Validation losses: 0.2969, 380.6536, 380.9505	Best Loss: 380.9505 (0)	Accuracy: 89.06%
7	Validation losses: 0.4131, 375.8265, 376.2396	Best Loss: 376.2396 (0)	Accuracy: 84.92%
8	Validation losses: 0.2955, 377.2946, 377.5901	Best Loss: 376.2396 (1)	Accuracy: 89.16%
9	Validation losses: 0.3204, 385.0414, 385.3618	Best Loss: 376.2396 (2)	Accuracy: 88.34%
10	Validation losses: 0.3015, 391.8799, 392.1813	Best

91	Validation losses: 0.9403, 285.8788, 286.8191	Best Loss: 286.8191 (0)	Accuracy: 91.00%
92	Validation losses: 0.9435, 285.8649, 286.8084	Best Loss: 286.8084 (0)	Accuracy: 91.04%
93	Validation losses: 0.9471, 285.8514, 286.7985	Best Loss: 286.7985 (0)	Accuracy: 91.04%
94	Validation losses: 0.9506, 285.8385, 286.7891	Best Loss: 286.7891 (0)	Accuracy: 91.02%
95	Validation losses: 0.9546, 285.8222, 286.7768	Best Loss: 286.7768 (0)	Accuracy: 91.06%
96	Validation losses: 0.9582, 285.8059, 286.7641	Best Loss: 286.7641 (0)	Accuracy: 91.06%
97	Validation losses: 0.9616, 285.7909, 286.7524	Best Loss: 286.7524 (0)	Accuracy: 91.04%
98	Validation losses: 0.9654, 285.7761, 286.7415	Best Loss: 286.7415 (0)	Accuracy: 91.06%
99	Validation losses: 0.9691, 285.7602, 286.7293	Best Loss: 286.7293 (0)	Accuracy: 91.06%
100	Validation losses: 0.9727, 285.7454, 286.7180	Best Loss: 286.7180 (0)	Accuracy: 91.06%
101	Validation losses: 0.9764, 285.7291, 286.7055	Best Loss: 286.7055 (0)	Accuracy: 91.06%
102	Vali

182	Validation losses: 1.0474, 285.4107, 286.4581	Best Loss: 286.4581 (0)	Accuracy: 91.02%
183	Validation losses: 1.0479, 285.4087, 286.4566	Best Loss: 286.4566 (0)	Accuracy: 91.02%
184	Validation losses: 1.0485, 285.4059, 286.4544	Best Loss: 286.4544 (0)	Accuracy: 91.02%
185	Validation losses: 1.0490, 285.4036, 286.4526	Best Loss: 286.4526 (0)	Accuracy: 91.04%
186	Validation losses: 1.0494, 285.4010, 286.4504	Best Loss: 286.4504 (0)	Accuracy: 91.04%
187	Validation losses: 1.0499, 285.3988, 286.4487	Best Loss: 286.4487 (0)	Accuracy: 91.04%
188	Validation losses: 1.0503, 285.3969, 286.4472	Best Loss: 286.4472 (0)	Accuracy: 91.04%
189	Validation losses: 1.0507, 285.3956, 286.4463	Best Loss: 286.4463 (0)	Accuracy: 91.04%
190	Validation losses: 1.0511, 285.3938, 286.4449	Best Loss: 286.4449 (0)	Accuracy: 91.04%
191	Validation losses: 1.0515, 285.3929, 286.4443	Best Loss: 286.4443 (0)	Accuracy: 91.04%
192	Validation losses: 1.0518, 285.3917, 286.4435	Best Loss: 286.4435 (0)	Accuracy: 91.04%

21	Validation losses: nan, nan, nan	Best Loss: 376.1238 (14)	Accuracy: 0.00%
22	Validation losses: nan, nan, nan	Best Loss: 376.1238 (15)	Accuracy: 0.00%
23	Validation losses: nan, nan, nan	Best Loss: 376.1238 (16)	Accuracy: 0.00%
24	Validation losses: nan, nan, nan	Best Loss: 376.1238 (17)	Accuracy: 0.00%
25	Validation losses: nan, nan, nan	Best Loss: 376.1238 (18)	Accuracy: 0.00%
26	Validation losses: nan, nan, nan	Best Loss: 376.1238 (19)	Accuracy: 0.00%
27	Validation losses: nan, nan, nan	Best Loss: 376.1238 (20)	Accuracy: 0.00%
Early stopping!
Total running time:	 2720
INFO:tensorflow:Restoring parameters from ./models/model_20240324025935.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    12.6099991798%
RANDOM SEED:   3 	LAMBDA_EXP:   -4.0
Now: 20240324034457
0	Validation losses: 0.4359, 415.2280, 415.6639	Best Loss: 415.6639 (0)	Accuracy: 85.24%
1	Validation losses: 0.3602, 407.412

3	Validation losses: 0.3499, 251.3929, 251.7428	Best Loss: 251.7428 (0)	Accuracy: 86.96%
4	Validation losses: 0.3162, 248.0233, 248.3395	Best Loss: 248.3395 (0)	Accuracy: 88.58%
5	Validation losses: 0.3259, 244.6786, 245.0045	Best Loss: 245.0045 (0)	Accuracy: 87.82%
6	Validation losses: 0.3449, 241.5020, 241.8470	Best Loss: 241.8470 (0)	Accuracy: 87.24%
7	Validation losses: 0.3205, 238.2010, 238.5215	Best Loss: 238.5215 (0)	Accuracy: 88.70%
8	Validation losses: 0.3026, 234.8508, 235.1533	Best Loss: 235.1533 (0)	Accuracy: 88.88%
9	Validation losses: 0.3030, 231.6459, 231.9489	Best Loss: 231.9489 (0)	Accuracy: 89.34%
10	Validation losses: 0.3338, 228.5801, 228.9139	Best Loss: 228.9139 (0)	Accuracy: 88.30%
11	Validation losses: 0.3268, 225.4740, 225.8008	Best Loss: 225.8008 (0)	Accuracy: 88.68%
12	Validation losses: 0.3054, 222.3303, 222.6357	Best Loss: 222.6357 (0)	Accuracy: 89.14%
13	Validation losses: 0.3391, 223.1187, 223.4578	Best Loss: 222.6357 (1)	Accuracy: 88.06%
14	Validation los

...

Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    9.9599993229%
RANDOM SEED:   2 	LAMBDA_EXP:   -5.2
Now: 20240325061618
0	Validation losses: 0.3997, 27.2941, 27.6938	Best Loss: 27.6938 (0)	Accuracy: 85.56%
1	Validation losses: 0.3556, 26.7033, 27.0589	Best Loss: 27.0589 (0)	Accuracy: 87.26%
2	Validation losses: 0.3514, 26.2823, 26.6338	Best Loss: 26.6338 (0)	Accuracy: 87.38%
3	Validation losses: 0.3563, 25.9252, 26.2816	Best Loss: 26.2816 (0)	Accuracy: 86.24%
4	Validation losses: 0.3187, 25.5819, 25.9006	Best Loss: 25.9006 (0)	Accuracy: 88.44%
5	Validation losses: 0.3388, 25.2424, 25.5812	Best Loss: 25.5812 (0)	Accuracy: 87.66%
6	Validation losses: 0.3061, 24.9224, 25.2285	Best Loss: 25.2285 (0)	Accuracy: 88.62%
7	Validation losses: 0.3302, 24.6131, 24.9434	Best Loss: 24.9434 (0)	Accuracy: 87.82%
8	Validation losses: 0.3015, 24.2661, 24.5676	Best Loss: 24.5676 (0)	Accuracy: 88.90%
9	Validati

10	Validation losses: 0.2895, 14.9958, 15.2852	Best Loss: 15.2852 (0)	Accuracy: 89.88%
11	Validation losses: 0.3129, 14.8395, 15.1525	Best Loss: 15.1525 (0)	Accuracy: 88.76%
12	Validation losses: 0.4765, 14.6525, 15.1291	Best Loss: 15.1291 (0)	Accuracy: 83.76%
13	Validation losses: 0.3332, 14.4568, 14.7901	Best Loss: 14.7901 (0)	Accuracy: 88.64%
14	Validation losses: 0.2963, 14.2150, 14.5113	Best Loss: 14.5113 (0)	Accuracy: 89.54%
15	Validation losses: 0.3092, 14.0538, 14.3629	Best Loss: 14.3629 (0)	Accuracy: 89.54%
16	Validation losses: 0.3221, 13.8889, 14.2110	Best Loss: 14.2110 (0)	Accuracy: 89.28%
17	Validation losses: 0.3369, 13.7332, 14.0701	Best Loss: 14.0701 (0)	Accuracy: 89.30%
18	Validation losses: 0.3155, 13.5585, 13.8740	Best Loss: 13.8740 (0)	Accuracy: 89.30%
19	Validation losses: nan, nan, nan	Best Loss: 13.8740 (1)	Accuracy: 0.00%
20	Validation losses: nan, nan, nan	Best Loss: 13.8740 (2)	Accuracy: 0.00%
21	Validation losses: nan, nan, nan	Best Loss: 13.8740 (3)	Accuracy

28	Validation losses: 0.4079, 12.5464, 12.9543	Best Loss: 12.9543 (0)	Accuracy: 90.28%
29	Validation losses: 0.3859, 12.5260, 12.9119	Best Loss: 12.9119 (0)	Accuracy: 90.16%
30	Validation losses: 0.4311, 12.4402, 12.8713	Best Loss: 12.8713 (0)	Accuracy: 90.48%
31	Validation losses: 0.4708, 12.4013, 12.8721	Best Loss: 12.8713 (1)	Accuracy: 90.48%
32	Validation losses: 0.5142, 12.3733, 12.8875	Best Loss: 12.8713 (2)	Accuracy: 90.24%
33	Validation losses: 0.5296, 12.3630, 12.8926	Best Loss: 12.8713 (3)	Accuracy: 90.82%
34	Validation losses: 0.5994, 12.3131, 12.9125	Best Loss: 12.8713 (4)	Accuracy: 90.96%
35	Validation losses: 0.6376, 12.3150, 12.9526	Best Loss: 12.8713 (5)	Accuracy: 90.32%
36	Validation losses: 0.6735, 12.2910, 12.9645	Best Loss: 12.8713 (6)	Accuracy: 90.70%
37	Validation losses: 0.7255, 12.2780, 13.0035	Best Loss: 12.8713 (7)	Accuracy: 90.64%
38	Validation losses: 0.7684, 12.2739, 13.0423	Best Loss: 12.8713 (8)	Accuracy: 90.70%
39	Validation losses: 0.7896, 12.2716, 13.0

14	Validation losses: 0.3997, 9.1428, 9.5424	Best Loss: 9.5424 (0)	Accuracy: 83.76%
15	Validation losses: nan, nan, nan	Best Loss: 9.5424 (1)	Accuracy: 0.00%
16	Validation losses: nan, nan, nan	Best Loss: 9.5424 (2)	Accuracy: 0.00%
17	Validation losses: nan, nan, nan	Best Loss: 9.5424 (3)	Accuracy: 0.00%
18	Validation losses: nan, nan, nan	Best Loss: 9.5424 (4)	Accuracy: 0.00%
19	Validation losses: nan, nan, nan	Best Loss: 9.5424 (5)	Accuracy: 0.00%
20	Validation losses: nan, nan, nan	Best Loss: 9.5424 (6)	Accuracy: 0.00%
21	Validation losses: nan, nan, nan	Best Loss: 9.5424 (7)	Accuracy: 0.00%
22	Validation losses: nan, nan, nan	Best Loss: 9.5424 (8)	Accuracy: 0.00%
23	Validation losses: nan, nan, nan	Best Loss: 9.5424 (9)	Accuracy: 0.00%
24	Validation losses: nan, nan, nan	Best Loss: 9.5424 (10)	Accuracy: 0.00%
25	Validation losses: nan, nan, nan	Best Loss: 9.5424 (11)	Accuracy: 0.00%
26	Validation losses: nan, nan, nan	Best Loss: 9.5424 (12)	Accuracy: 0.00%
27	Validation losses: nan

19	Validation losses: nan, nan, nan	Best Loss: 5.9250 (1)	Accuracy: 0.00%
20	Validation losses: nan, nan, nan	Best Loss: 5.9250 (2)	Accuracy: 0.00%
21	Validation losses: nan, nan, nan	Best Loss: 5.9250 (3)	Accuracy: 0.00%
22	Validation losses: nan, nan, nan	Best Loss: 5.9250 (4)	Accuracy: 0.00%
23	Validation losses: nan, nan, nan	Best Loss: 5.9250 (5)	Accuracy: 0.00%
24	Validation losses: nan, nan, nan	Best Loss: 5.9250 (6)	Accuracy: 0.00%
25	Validation losses: nan, nan, nan	Best Loss: 5.9250 (7)	Accuracy: 0.00%
26	Validation losses: nan, nan, nan	Best Loss: 5.9250 (8)	Accuracy: 0.00%
27	Validation losses: nan, nan, nan	Best Loss: 5.9250 (9)	Accuracy: 0.00%
28	Validation losses: nan, nan, nan	Best Loss: 5.9250 (10)	Accuracy: 0.00%
29	Validation losses: nan, nan, nan	Best Loss: 5.9250 (11)	Accuracy: 0.00%
30	Validation losses: nan, nan, nan	Best Loss: 5.9250 (12)	Accuracy: 0.00%
31	Validation losses: nan, nan, nan	Best Loss: 5.9250 (13)	Accuracy: 0.00%
32	Validation losses: nan, nan, na